In [2]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import train_test_split
from tqdm import tqdm
from sklearn.metrics import mean_squared_error
import scipy as sp

In [3]:
NUM_RANDOM_SEEDS = 5
MODEL_CLASS = "lr"
EPSILON = 0.05
NUM_PERTURBED_MODELS = 100

In [4]:
data_source = "data/obermeyer/obermeyer_data_cleaned.csv"
prediction_output = "predictions/obermeyer/model_weight_perturbation_"+MODEL_CLASS+".csv"
target_variable = "cost_t"
features = ['dem_female', 'dem_age_band_18-24_tm1', 'dem_age_band_25-34_tm1', 'dem_age_band_35-44_tm1', 'dem_age_band_45-54_tm1',
            'dem_age_band_55-64_tm1', 'dem_age_band_65-74_tm1', 'dem_age_band_75+_tm1', 'hypertension_elixhauser_tm1', 'cost_dialysis_tm1',
            'cost_emergency_tm1', 'cost_home_health_tm1', 'cost_ip_medical_tm1', 'cost_ip_surgical_tm1', 'cost_laboratory_tm1',
            'cost_op_primary_care_tm1', 'cost_op_specialists_tm1', 'cost_op_surgery_tm1', 'cost_other_tm1', 'cost_pharmacy_tm1',
            'cost_physical_therapy_tm1', 'cost_radiology_tm1', 'gagne_sum_tm1']
other_variables = ['person_id', 'qualified_cost_25', 'qualified_cost_75', 'qualified_cost_50', 'qualified_gagne_1', 'qualified_gagne_2', 'qualified_gagne_3']

In [7]:
df = pd.read_csv(data_source)
X = df[features+other_variables]
y = df[target_variable]

In [8]:
def get_baseline_model_coefficients(X_train, y_train):
    if MODEL_CLASS == "lr":
        model = LinearRegression()
    model.fit(X_train, y_train)
    return model.coef_

In [9]:
def get_perturbed_weights(baseline_weights, epsilon):
    random_vector = np.random.randn(*baseline_weights.shape)
    random_vector = random_vector / np.linalg.norm(random_vector)
    perturbation = np.random.uniform(0, epsilon) * random_vector
    w = baseline_weights + perturbation
    return w

In [10]:
def get_predictions_columns(X_test, baseline_weights, epsilon):
    predictions = []
    columns = []
    for i in tqdm(range(NUM_PERTURBED_MODELS)):
        perturbed_weights = get_perturbed_weights(baseline_weights, epsilon)
        predictions.append(np.dot(X_test, perturbed_weights))
        columns.append(f'm_{i+1}')
    return predictions, columns

In [14]:
output = []
for random_seed in range(NUM_RANDOM_SEEDS):
    print("random seed", random_seed)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=random_seed)
    
    cost_threshold_25 = X_test['qualified_cost_25'].to_numpy()
    cost_threshold_50 = X_test['qualified_cost_50'].to_numpy()
    cost_threshold_75 = X_test['qualified_cost_75'].to_numpy()
    gagne_threshold_1 = X_test['qualified_gagne_1'].to_numpy()
    gagne_threshold_2 = X_test['qualified_gagne_2'].to_numpy()
    gagne_threshold_3 = X_test['qualified_gagne_3'].to_numpy()
    test_idx = X_test['person_id'].to_numpy()

    X_train = X_train.drop(columns=other_variables).to_numpy()
    X_train = sp.linalg.orth(X_train)
    y_train = y_train.to_numpy()
    X_test = X_test.drop(columns=other_variables).to_numpy()
    X_test = sp.linalg.orth(X_test)
    y_test = y_test.to_numpy()

    baseline_weights = get_baseline_model_coefficients(X_train, y_train)
    predictions, columns = get_predictions_columns(X_test, baseline_weights, EPSILON)
    
    
    predictions_df = pd.DataFrame(predictions).transpose()
    predictions_df.columns=columns
    predictions_df["y"] = y_test
    predictions_df["person_id"] = test_idx 
    predictions_df["seed"] = random_seed
    
    predictions_df['qualified_cost_25'] = cost_threshold_25
    predictions_df['qualified_cost_50'] = cost_threshold_50
    predictions_df['qualified_cost_75'] = cost_threshold_75
    predictions_df['qualified_gagne_1'] = gagne_threshold_1
    predictions_df['qualified_gagne_2'] = gagne_threshold_2
    predictions_df['qualified_gagne_3'] = gagne_threshold_3

    output.append(predictions_df)
output = pd.concat(output)

random seed 0


100%|██████████| 100/100 [00:00<00:00, 371.09it/s]


random seed 1


100%|██████████| 100/100 [00:00<00:00, 347.43it/s]


random seed 2


100%|██████████| 100/100 [00:00<00:00, 374.07it/s]


random seed 3


100%|██████████| 100/100 [00:00<00:00, 355.26it/s]


random seed 4


100%|██████████| 100/100 [00:00<00:00, 356.78it/s]


In [15]:
output.head()

,m_1,m_2,m_3,m_4,m_5,m_6,m_7,m_8,m_9,m_10,...,m_100,y,person_id,seed,qualified_cost_25,qualified_cost_50,qualified_cost_75,qualified_gagne_1,qualified_gagne_2,qualified_gagne_3
0,0.007089,0.007156,0.007029,0.007102,0.006922,0.007039,0.007221,0.007198,0.007105,0.007117,...,0.006859,0.009628,2545,0,0,1,1,1,0,0
1,0.010493,0.010462,0.010987,0.010489,0.010355,0.010560,0.010672,0.010709,0.010589,0.010639,...,0.010455,0.004905,8198,0,0,0,1,1,1,0
2,0.039912,0.039877,0.040471,0.039823,0.040040,0.040202,0.039607,0.040148,0.039583,0.039899,...,0.039697,0.009446,46461,0,0,1,1,0,0,0
3,-0.006416,-0.006375,-0.006342,-0.006257,-0.006310,-0.006390,-0.006002,-0.006293,-0.006223,-0.006374,...,-0.006495,0.002361,30620,0,0,0,1,0,0,0
4,-0.007824,-0.007816,-0.008165,-0.007837,-0.008462,-0.007940,-0.007493,-0.007855,-0.007970,-0.007930,...,-0.008012,0.003996,47418,0,0,0,1,1,1,0


In [16]:
output.info()

<class 'pandas.core.frame.DataFrame'>
Index: 80495 entries, 0 to 16098
Columns: 109 entries, m_1 to qualified_gagne_3
dtypes: float64(101), int64(8)
memory usage: 67.6 MB


In [17]:
output.to_csv(prediction_output, index=False)